In [ ]:
from IPython.display import HTML
HTML(filename='/Users/hubatsl/Desktop/SPT/Us/SPT/Python/hide_code.html')

# Off rate databank for pkc-ts experiment

In [ ]:
from ipywidgets import interact
from matplotlib import pyplot as plt
from matplotlib import mlab 
import numpy as np
import pandas as pd
import os
import seaborn as sns
import sys
import tifffile
import time
# Add path to python modules
sys.path.append(os.getcwd()[0:-11]+'src/') 
from MovieTracks import (DiffusionFitter, OffRateFitter, ParticleFinder)
# Plotting becomes part of the notebook
%matplotlib inline

**Run thresholding on movie movieNo, read and write threshold from excel file**

In [ ]:
xls_file = '/Users/hubatsl/Desktop/Florent/OffRateFigureFlorent/OffRateTS.xlsx'
xls_db = pd.read_excel(xls_file)
movieNo = 0

if np.isnan(xls_db['Threshold'][movieNo]):
    xls_db.loc[movieNo, 'Threshold']=1000
thresh = int(xls_db['Threshold'][movieNo])
if np.isnan(xls_db['StartFrame'][movieNo]):
    xls_db.loc[movieNo, 'StartFrame'] = 0
startFrame = int(xls_db.loc[movieNo, 'StartFrame'])
if np.isnan(xls_db['MaxSize'][movieNo]):
    xls_db.loc[movieNo, 'MaxSize'] = 0
maxSize = int(xls_db.loc[movieNo, 'MaxSize'])

def adj_thresh(calibFrame, thresh, maxSize, startFrame):
    calibFrame = int(calibFrame)
    thresh = int(thresh)
    startFrame = int(startFrame)
    maxSize = int(maxSize)
    if maxSize is not 0: 
        maxSize = int(maxSize)
        o = OffRateFitter(xls_db['Folder'][movieNo], thresh, pixelSize=0.120,
                          maxsize=maxSize/10000, startFrame=startFrame, featSize=5)
    else:
        o = OffRateFitter(xls_db['Folder'][movieNo], thresh, pixelSize=0.120,
                          maxsize=None, startFrame=startFrame)
    print(xls_db['Folder'][movieNo])
    o.plot_calibration(calibrationFrame=calibFrame)
    xls_db.loc[movieNo, 'StartFrame'] = startFrame
    xls_db.loc[movieNo, 'Threshold'] = thresh
    xls_db.loc[movieNo, 'MaxSize'] = maxSize
    xls_db.to_excel(xls_file)

interact(adj_thresh, thresh=str(thresh), calibFrame='0', maxSize=str(maxSize),
         startFrame=str(startFrame), __manual=True)

**Read time stamps for each movie, write to excel file**

In [ ]:
xls_db['IntervalReal'] = 'nan'
xls_db['ExposureReal'] = 'nan'
xls_db['Length'] = 'nan'
for i, path in enumerate(xls_db['Folder']):
    p = ParticleFinder(path)
    xls_db.loc[i, 'IntervalReal'] = p.timestep
    xls_db.loc[i, 'ExposureReal'] = p.exposure
    xls_db.loc[i, 'Length'] = len(p.frames)
xls_db.to_excel(xls_file)

In [ ]:
offRates = []
onRates = []
bleachingRates = []
for i in range(len(xls_db['Folder'])):
    fol = xls_db.loc[i, 'Folder']
    threshold = xls_db.loc[i, 'Threshold']
    maxsize = xls_db.loc[i, 'MaxSize']
    if int(maxsize) is 0:
        maxsize=None
    else: maxsize = float(maxsize)/10000
    o = OffRateFitter(filePath=fol, threshold=threshold, maxsize=maxsize,
                      parallel=False, pixelSize=0.12,
                      saveFigs=True, showFigs=True, autoMetaDataExtract=True,
                      featSize=5)
    o.analyze()
    o.plot_calibration(calibrationFrame=10)
    o.fit_offRate([1, 2, 3, 4, 5, 6])
    o.append_output_to_csv('/Users/hubatsl/Desktop/Florent/OffRateFigureFlorent/Off_ts.csv', o.gData())
#     input("Press Enter to continue...")

In [ ]:
%qtconsole

In [ ]:
df = o.features

In [ ]:
%config InlineBackend.figure_format = 'retina'
from sklearn import mixture
import matplotlib.pyplot
import matplotlib.mlab
import numpy as np

clf = mixture.GaussianMixture(n_components=2, covariance_type='full')
data = df.loc[df['frame']==0].mass.tolist()
print(len(data))
data = [[x] for x in data if x <10000]
print(len(data))
clf.fit(data)
m1, m2 = clf.means_
w1, w2 = clf.weights_
c1, c2 = clf.covariances_
data = [item for sublist in data for item in sublist]

rangeMin = 0
rangeMax = 10000
h = matplotlib.pyplot.hist(data, bins=50, range=(rangeMin, rangeMax), normed=True);
plt.plot(range(rangeMin, rangeMax, 100),
         w1*mlab.normpdf(range(rangeMin, rangeMax, 100), m1, np.sqrt(c1[0]))+
         w2*mlab.normpdf(range(rangeMin, rangeMax, 100), m2, np.sqrt(c2[0])))
# plt.plot(range(rangeMin, rangeMax, 100), 
#          w1*mlab.normpdf(range(rangeMin, rangeMax, 100), m1, np.sqrt(c1[0])))
# plt.plot(range(rangeMin, rangeMax, 100),
#          w2*mlab.normpdf(range(rangeMin, rangeMax, 100), m2, np.sqrt(c2[0])))
plt.show()
print(w1/(w1+w2))

In [ ]:
%config InlineBackend.figure_format = 'retina'
from sklearn import mixture
import matplotlib.pyplot
import matplotlib.mlab
import numpy as np

clf = mixture.GaussianMixture(n_components=2, covariance_type='full')
data = np.random.randn(1000)
data = [[x] for x in data if x <0]
clf.fit(data)
m1, m2 = clf.means_
w1, w2 = clf.weights_
c1, c2 = clf.covariances_
data = [item for sublist in data for item in sublist]

rangeMin = int(np.floor(np.min(data)))
rangeMax = int(np.ceil(np.max(data)))
h = matplotlib.pyplot.hist(data, range=(rangeMin, rangeMax), normed=True);
plt.plot(np.linspace(rangeMin, rangeMax),
         w1*mlab.normpdf(np.linspace(rangeMin, rangeMax), m1, np.sqrt(c1[0]))+
         w2*mlab.normpdf(np.linspace(rangeMin, rangeMax), m2, np.sqrt(c2[0])))
# plt.plot(range(rangeMin, rangeMax, 100), 
#          w1*mlab.normpdf(range(rangeMin, rangeMax, 100), m1, np.sqrt(c1[0])))
# plt.plot(range(rangeMin, rangeMax, 100),
#          w2*mlab.normpdf(range(rangeMin, rangeMax, 100), m2, np.sqrt(c2[0])))
plt.show()
# print(w1/(w1+w2))

In [ ]:
print(m1)
print(m2)
print(w1)
print(w2)

In [ ]:
from sklearn import mixture
import matplotlib.pyplot
import matplotlib.mlab
import numpy as np

clf = mixture.GaussianMixture(n_components=1, covariance_type='full', means_init=[[0]])
data = np.random.randn(1000)
data = [[x] for x in data if x <0]
# data = [[x] for x in data]
clf.fit(data)
data = [item for sublist in data for item in sublist]
rangeMin = int(np.floor(np.min(data)))
rangeMax = int(np.ceil(np.max(data)))
h = matplotlib.pyplot.hist(data, range=(rangeMin, rangeMax), normed=True);
plt.plot(np.linspace(rangeMin, rangeMax),
         mlab.normpdf(np.linspace(rangeMin, rangeMax),
                      clf.means_, np.sqrt(clf.covariances_[0]))[0])

In [ ]:
import matplotlib.pyplot
import matplotlib.mlab
import numpy as np

def gaus(bin_mid, params):
    a1, x01, sigma1 = params    
    return a1*np.exp(-(bin_mid-x01)**2/(2*sigma1**2))

def gaus_obj(params, bin_mid, data):
    return np.sum((data-gaus(bin_mid, params))**2)**0.5

data = np.random.randn(500)
data = data[data<0]
rangeMin = int(np.floor(np.min(data)))
rangeMax = int(np.ceil(np.max(data)))
h = matplotlib.pyplot.hist(data, range=(rangeMin, rangeMax), normed=True)
data = h[0]
bin_mid = h[1][0:-1]+np.mean(np.diff(h[1]))/2
opt_result = differential_evolution(gaus_obj, [(0.7, 0.9), (-0.1, 0.1), (0.9, 1.1)],
                                    args=(gaus(bin_mid, [0.8, 0, 1]), [bin_mid]), strategy='randtobest1exp')
plt.plot(bin_mid, gaus(bin_mid, opt_result.x), color = 'c')
plt.plot(bin_mid, gaus(bin_mid, [0.8, 0, 1]))
print(gaus_obj(opt_result.x, bin_mid, data))
print(gaus_obj([0.8, 0, 1], bin_mid, data))
print(opt_result.x)
plt.plot(bin_mid, data)

In [ ]:
params = (2, 3, 7, 8, 9, 10, 44, -1, 2, 26, 1, -2, 0.5)
def f1(z, *params):
    x, y = z
    a, b, c, d, e, f, g, h, i, j, k, l, scale = params
    return (a * x**2 + b * x * y + c * y**2 + d*x + e*y + f)
def f2(z, *params):
    x, y = z
    a, b, c, d, e, f, g, h, i, j, k, l, scale = params
    return (-g*np.exp(-((x-h)**2 + (y-i)**2) / scale))
def f3(z, *params):
    x, y = z
    a, b, c, d, e, f, g, h, i, j, k, l, scale = params
    return (-j*np.exp(-((x-k)**2 + (y-l)**2) / scale))
def f(z, *params):
     return f1(z, *params) + f2(z, *params) + f3(z, *params)
rranges = (slice(-4, 4, 0.25), slice(-4, 4, 0.25))
from scipy import optimize
resbrute = optimize.brute(f, rranges, args=params, full_output=True,
                           finish=optimize.fmin)
print(resbrute[0])  # global minimum
print(resbrute[1])  # function value at global minimum

In [ ]:
from scipy.optimize import differential_evolution

def func(parameters, *data):

    #we have 3 parameters which will be passed as parameters and
    #"experimental" x,y which will be passed as data

    a,b,c = parameters
    x,y = data

    result = 0

    for i in range(len(x)):
        result += (a*x[i]**2 + b*x[i]+ c - y[i])**2

    return result**0.5

if __name__ == '__main__':
    #initial guess for variation of parameters
    #             a            b            c
    bounds = [(-10, 10), (-10, 10), (-10, 10)]

    #producing "experimental" data 
    x = [i for i in range(6)]
    y = [x**2 for x in x]

    #packing "experimental" data into args
    args = (x,y)

    result = differential_evolution(func, bounds, args=args)
    print(result.x)

In [ ]:
X = df.loc[df['frame']==1].mass.tolist()
gmm = mixture.GaussianMixture(n_components=3, covariance_type='full').fit(X)

In [ ]:
X = np.r_[np.dot(np.random.randn(n_samples, 1), C)];X

In [ ]:
import itertools

import numpy as np
from scipy import linalg
import matplotlib.pyplot as plt
import matplotlib as mpl

from sklearn import mixture

color_iter = itertools.cycle(['navy', 'c', 'cornflowerblue', 'gold',
                              'darkorange'])


def plot_results(X, Y_, means, covariances, index, title):
    splot = plt.subplot(2, 1, 1 + index)
    for i, (mean, covar, color) in enumerate(zip(
            means, covariances, color_iter)):
        v, w = linalg.eigh(covar)
        v = 2. * np.sqrt(2.) * np.sqrt(v)
        u = w[0] / linalg.norm(w[0])
        # as the DP will not use every component it has access to
        # unless it needs it, we shouldn't plot the redundant
        # components.
        if not np.any(Y_ == i):
            continue
        plt.scatter(X[Y_ == i, 0], X[Y_ == i, 1], .8, color=color)

        # Plot an ellipse to show the Gaussian component
        angle = np.arctan(u[1] / u[0])
        angle = 180. * angle / np.pi  # convert to degrees
        ell = mpl.patches.Ellipse(mean, v[0], v[1], 180. + angle, color=color)
        ell.set_clip_box(splot.bbox)
        ell.set_alpha(0.5)
        splot.add_artist(ell)

    plt.xlim(-9., 5.)
    plt.ylim(-3., 6.)
    plt.xticks(())
    plt.yticks(())
    plt.title(title)


# Number of samples per component
n_samples = 500

# Generate random sample, two components
np.random.seed(0)
C = np.array([[0.1]])
X = np.r_[np.dot(np.random.randn(n_samples, 1), C)]

# Fit a Gaussian mixture with EM using five components
gmm = mixture.GaussianMixture(n_components=5, covariance_type='full').fit(X)
plot_results(X, gmm.predict(X), gmm.means_, gmm.covariances_, 0,
             'Gaussian Mixture')

# Fit a Dirichlet process Gaussian mixture using five components
dpgmm = mixture.BayesianGaussianMixture(n_components=5,
                                        covariance_type='full').fit(X)
plot_results(X, dpgmm.predict(X), dpgmm.means_, dpgmm.covariances_, 1,
             'Bayesian Gaussian Mixture with a Dirichlet process prior')

plt.show()

In [ ]:
plt.hist(X); plt.show()
gmm.means_